#ETL com arquivos legados -> CSV + Postgres

Extração

In [6]:
import pandas as pd

clientes = pd.read_csv("dataset_semana3/clientes_legado.csv")
pedidos = pd.read_csv("dataset_semana3/pedidos_legado.csv")

print(clientes.head())
print(pedidos.head())

   id_cliente   nome_completo                     email          cidade
0         101       Ana Souza       ana.souza@email.com       São Paulo
1         102      Bruno Lima      bruno.lima@email.com  Rio de Janeiro
2         103    Carla Mendes    carla.mendes@email.com  Belo Horizonte
3         104  Diego Ferreira  diego.ferreira@email.com    Porto Alegre
4         105   Elisa Martins   elisa.martins@email.com        Curitiba
   id_pedido  cliente_ref  valor_total data_compra
0       5001          101        250.0  2023-01-10
1       5002          101        320.0  2023-03-15
2       5003          102        150.0  2023-02-05
3       5004          102        180.0  2023-04-12
4       5005          102        210.0  2023-06-20


Tranformação

In [5]:
clientes = clientes.rename(columns={
    "id_cliente": "id",
    "nome_completo": "nome"
})

pedidos = pedidos.rename(columns={
    "id_pedido": "id",
    "cliente_ref": "cliente_id",
    "valor_total": "valor",
    "data_compra": "data_pedido"
})
# Conversão de tipos
pedidos["data_pedido"] = pd.to_datetime(pedidos["data_pedido"])

# Verificações de integridade 
print(clientes["email"].duplicated().sum())
pedidos_invalidos = pedidos[~pedidos["cliente_id"].isin(clientes["id"])]
print(pedidos_invalidos)

0
Empty DataFrame
Columns: [id, cliente_id, valor, data_pedido]
Index: []


Carga (Load)

In [8]:
from conexao import get_engine

engine = get_engine()

# Carregar clientes e pedidos no Postgres
clientes.to_sql("clientes", engine, schema="etl_course", if_exists="replace", index=False)
pedidos.to_sql("pedidos", engine, schema="etl_course", if_exists="replace", index=False)

11